In [1]:
#imports
import pandas as pd
import numpy as np
from env import get_db_url
import os
import acquire
from sklearn import metrics

In [2]:
#using acquire code and file to call in curriculum_log data:
df = acquire.get_curriculum_data()
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 900223 entries, 0 to 900222
Data columns (total 9 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   date        900223 non-null  object 
 1   time        900223 non-null  object 
 2   path        900222 non-null  object 
 3   user_id     900223 non-null  int64  
 4   ip          900223 non-null  object 
 5   name        847330 non-null  object 
 6   start_date  847330 non-null  object 
 7   end_date    847330 non-null  object 
 8   program_id  847330 non-null  float64
dtypes: float64(1), int64(1), object(7)
memory usage: 68.7+ MB


,date,time,path,user_id,ip,name,start_date,end_date,program_id
0,2018-01-26,09:55:03,/,1,97.105.19.61,Hampton,2015-09-22,2016-02-06,1.0
1,2018-01-26,09:56:02,java-ii,1,97.105.19.61,Hampton,2015-09-22,2016-02-06,1.0
2,2018-01-26,09:56:05,java-ii/object-oriented-programming,1,97.105.19.61,Hampton,2015-09-22,2016-02-06,1.0
3,2018-01-26,09:56:06,slides/object_oriented_programming,1,97.105.19.61,Hampton,2015-09-22,2016-02-06,1.0
4,2018-01-26,09:56:24,javascript-i/conditionals,2,97.105.19.61,Teddy,2018-01-08,2018-05-17,2.0


In [3]:
df.index

Int64Index([     0,      1,      2,      3,      4,      5,      6,      7,
                 8,      9,
            ...
            900213, 900214, 900215, 900216, 900217, 900218, 900219, 900220,
            900221, 900222],
           dtype='int64', length=900223)

In [4]:
df.columns

Index(['date', 'time', 'path', 'user_id', 'ip', 'name', 'start_date',
       'end_date', 'program_id'],
      dtype='object')

In [5]:
#looking at the stats numbers for the dataset:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
user_id,900223.0,458.825707,249.296767,1.0,269.0,475.0,660.0,981.0
program_id,847330.0,2.086004,0.388231,1.0,2.0,2.0,2.0,4.0


### Take aways:
- `date`/ `time` / `start_date` and `end_date` needs to be run through `pd.to_datetime`
- `date` could also become new index
- it could be helpful to create a `program_name` column
- setting `ip` as float will allow boxplots and other charts for anomaly checks.

In [6]:
#passing df through pd.to_datetime for date and time format/dtype for all date/time columns
df.date = pd.to_datetime(df.date)
df.start_date = pd.to_datetime(df.start_date)
df.end_date = pd.to_datetime(df.end_date)
#setting date as the index
df = df.set_index('date')

In [7]:
#checking dtypes:
df.dtypes

time                  object
path                  object
user_id                int64
ip                    object
name                  object
start_date    datetime64[ns]
end_date      datetime64[ns]
program_id           float64
dtype: object

In [8]:
#checking new index:
df.index

DatetimeIndex(['2018-01-26', '2018-01-26', '2018-01-26', '2018-01-26',
               '2018-01-26', '2018-01-26', '2018-01-26', '2018-01-26',
               '2018-01-26', '2018-01-26',
               ...
               '2021-04-21', '2021-04-21', '2021-04-21', '2021-04-21',
               '2021-04-21', '2021-04-21', '2021-04-21', '2021-04-21',
               '2021-04-21', '2021-04-21'],
              dtype='datetime64[ns]', name='date', length=900223, freq=None)

In [9]:
#adding in datetime columns to find stats values:
df.describe(datetime_is_numeric=True)

,user_id,start_date,end_date,program_id
count,900223.000000,847330,847330,847330.000000
mean,458.825707,2018-12-24 09:55:56.790861056,2019-05-11 21:08:19.869472256,2.086004
min,1.000000,2014-02-04 00:00:00,2014-02-04 00:00:00,1.000000
25%,269.000000,2018-05-29 00:00:00,2018-10-11 00:00:00,2.000000
50%,475.000000,2019-07-15 00:00:00,2019-12-11 00:00:00,2.000000
75%,660.000000,2020-03-23 00:00:00,2020-08-20 00:00:00,2.000000
max,981.000000,2021-04-12 00:00:00,2021-10-01 00:00:00,4.000000
std,249.296767,NaN,NaN,0.388231


_______________________________________________

### Looking at nulls and anomalies:

In [10]:
#out of the 90,223 rows and 9 columns, what nulls are in the data?
df.isnull().sum()

time              0
path              1
user_id           0
ip                0
name          52893
start_date    52893
end_date      52893
program_id    52893
dtype: int64

#### Checking anomalies on ip addresses 

In [11]:
#looking at ip column for outliers:
df.ip.value_counts()

97.105.19.58       284579
97.105.19.61        61662
192.171.117.210      9515
71.150.217.33        6791
76.185.145.231       4754
                    ...  
172.58.109.140          1
99.203.213.59           1
99.203.212.173          1
173.239.198.246         1
72.181.124.244          1
Name: ip, Length: 5531, dtype: int64

In [12]:
#defining a function to create a temp df for anomaly counts/frequencies:
def value_counts_and_frequencies(s: pd.Series, dropna=True) -> pd.DataFrame:
    return pd.merge(
    s.value_counts(dropna=False).rename('count'),
    s.value_counts(dropna=False, normalize=True).rename('proba'),
    left_index=True,
    right_index=True,
    )

ip_df = value_counts_and_frequencies(df.ip)
ip_df.head()

,count,proba
97.105.19.58,284579,0.316121
97.105.19.61,61662,0.068496
192.171.117.210,9515,0.010570
71.150.217.33,6791,0.007544
76.185.145.231,4754,0.005281


In [13]:
#looking at ip addresses that have the lowest frequencies occuring
df[df.ip == '172.58.109.140']

,time,path,user_id,ip,name,start_date,end_date,program_id
date,,,,,,,,
2020-01-04,18:54:11,/,419,172.58.109.140,Betelgeuse,2019-05-28,2019-10-08,2.0


In [14]:
#looking at ip addresses that have the lowest frequencies occuring
df[df.ip == '99.203.213.59']

,time,path,user_id,ip,name,start_date,end_date,program_id
date,,,,,,,,
2020-10-21,21:38:46,/,502,99.203.213.59,Staff,2014-02-04,2014-02-04,2.0


In [15]:
#looking at ip addresses that have the lowest frequencies occuring
df[df.ip == '72.181.124.244']

,time,path,user_id,ip,name,start_date,end_date,program_id
date,,,,,,,,
2018-09-18,19:04:09,/,88,72.181.124.244,NaN,NaT,NaT,NaN


#### interesting find/note about some of the nulls:

In [16]:
#This last user_id had nulls. When following it, this shows that possible nulls might be when students are
#going through their prework phase at Codeup...good to know for another time...
df[df.user_id == 88]

,time,path,user_id,ip,name,start_date,end_date,program_id
date,,,,,,,,
2018-02-05,21:40:45,index.html,88,172.5.98.113,NaN,NaT,NaT,NaN
2018-02-17,20:11:52,index.html,88,72.181.108.134,NaN,NaT,NaT,NaN
2018-02-17,20:12:09,prework/getting-started,88,72.181.108.134,NaN,NaT,NaT,NaN
2018-02-17,20:12:14,prework/fundamentals,88,72.181.108.134,NaN,NaT,NaT,NaN
2018-02-17,20:12:20,prework/vocab/decimal-number-system,88,72.181.108.134,NaN,NaT,NaT,NaN
...,...,...,...,...,...,...,...,...
2021-01-11,15:04:08,content/html-css/elements.html,88,136.50.6.131,Glacier,2015-06-05,2015-10-06,1.0
2021-01-11,15:04:09,content/html-css/forms.html,88,136.50.6.131,Glacier,2015-06-05,2015-10-06,1.0
2021-01-29,23:32:23,/,88,136.50.6.131,Glacier,2015-06-05,2015-10-06,1.0


In [38]:
#possible webscraper
df[df.user_id == 48]

,time,path,user_id,ip,name,start_date,end_date,program_id
date,,,,,,,,
2018-01-26,16:46:16,/,48,97.105.19.61,NaN,NaT,NaT,NaN
2018-01-26,16:46:24,spring/extra-features/form-validation,48,97.105.19.61,NaN,NaT,NaT,NaN
2018-01-26,17:54:24,/,48,97.105.19.61,NaN,NaT,NaT,NaN
2018-01-26,18:32:03,/,48,97.105.19.61,NaN,NaT,NaT,NaN
2018-01-26,18:32:17,mysql/relationships/joins,48,97.105.19.61,NaN,NaT,NaT,NaN
...,...,...,...,...,...,...,...,...
2020-04-08,16:42:09,javascript-i/introduction/primitive-types,48,216.177.189.26,NaN,NaT,NaT,NaN
2020-05-28,10:14:25,/,48,216.177.189.26,NaN,NaT,NaT,NaN
2020-05-28,10:14:30,jquery,48,216.177.189.26,NaN,NaT,NaT,NaN


In [37]:

df[df.name == 'Olympic']

,time,path,user_id,ip,name,start_date,end_date,program_id
date,,,,,,,,
2018-02-15,20:26:49,index.html,102,206.109.5.6,Olympic,2017-02-06,2017-05-25,1.0
2018-02-15,20:26:54,content/php_i,102,206.109.5.6,Olympic,2017-02-06,2017-05-25,1.0
2018-02-15,20:26:54,content/php_i/gitbook/images/favicon.ico,102,206.109.5.6,Olympic,2017-02-06,2017-05-25,1.0
2018-02-15,20:26:58,content/php_ii,102,206.109.5.6,Olympic,2017-02-06,2017-05-25,1.0
2018-02-15,20:26:58,content/php_ii/gitbook/images/favicon.ico,102,206.109.5.6,Olympic,2017-02-06,2017-05-25,1.0
...,...,...,...,...,...,...,...,...
2021-03-31,14:27:50,content/html-css,210,71.238.72.160,Olympic,2017-02-06,2017-05-25,1.0
2021-03-31,15:01:26,content/php_iv,210,71.238.72.160,Olympic,2017-02-06,2017-05-25,1.0
2021-03-31,15:01:31,content/javascript,210,71.238.72.160,Olympic,2017-02-06,2017-05-25,1.0


#### Next steps: 
- Turning `ip` into int?? (UPDATE: not needed. can be explored w/o ...verified by Ryan O.
- Handling nulls

### What is needed?
- Most frequent lessons logged into (per program)
    - Lesson counts
    - Program id/name
- Most/least frequent lessons logged into (per cohort)
    - Lesson log counts
    - name (cohort)
- Active students that do not engage with curriculum?
    - user_id frequency counts
    - current dates of cohorts
    - ip addresses?
- Suspicious activity and any webscrapping happening?
    - time (looking at odd hours of day)
    - ip addresses and no cohorts attached
    - user_id?
- 2019 dataframe showing shutoff of cross program curriculum access
    - program id and path page counts (?)
    - date (only 2019)
- Topics (path) alumni are still using?
    - path frequencies
    - anything > end_date of cohorts
- Least accessed lessons (overall)
    - path frequencies
- Anything else??


## Looking at Nulls:

In [17]:
#checking nulls:
df.isnull().sum()

time              0
path              1
user_id           0
ip                0
name          52893
start_date    52893
end_date      52893
program_id    52893
dtype: int64

In [25]:
#finding any missing names of cohorts (any possible scrapers?)
df[df.isnull().any(axis=1)]

,time,path,user_id,ip,name,start_date,end_date,program_id
date,,,,,,,,
2018-01-26,16:46:16,/,48,97.105.19.61,NaN,NaT,NaT,NaN
2018-01-26,16:46:24,spring/extra-features/form-validation,48,97.105.19.61,NaN,NaT,NaT,NaN
2018-01-26,17:54:24,/,48,97.105.19.61,NaN,NaT,NaT,NaN
2018-01-26,18:32:03,/,48,97.105.19.61,NaN,NaT,NaT,NaN
2018-01-26,18:32:17,mysql/relationships/joins,48,97.105.19.61,NaN,NaT,NaT,NaN
...,...,...,...,...,...,...,...,...
2021-04-21,12:49:00,javascript-ii,717,136.50.102.126,NaN,NaT,NaT,NaN
2021-04-21,12:49:02,javascript-ii/es6,717,136.50.102.126,NaN,NaT,NaT,NaN
2021-04-21,12:51:27,javascript-ii/map-filter-reduce,717,136.50.102.126,NaN,NaT,NaT,NaN


In [33]:
#looking through mid-sections of data
df[df.isnull().any(axis=1)][100:500]

,time,path,user_id,ip,name,start_date,end_date,program_id
date,,,,,,,,
2018-01-30,17:27:45,slides/arrays,61,173.173.114.158,NaN,NaT,NaT,NaN
2018-01-30,17:38:28,java-i/control-statements-and-loops,61,173.173.114.158,NaN,NaT,NaT,NaN
2018-01-30,17:38:30,slides/control_statements_and_loops,61,173.173.114.158,NaN,NaT,NaT,NaN
2018-01-30,17:40:33,/,48,66.69.62.219,NaN,NaT,NaT,NaN
2018-01-30,17:40:41,html-css/css-ii/bootstrap-grid-system,48,66.69.62.219,NaN,NaT,NaT,NaN
...,...,...,...,...,...,...,...,...
2018-02-10,23:42:48,javascript-i/conditionals,74,70.117.6.133,NaN,NaT,NaT,NaN
2018-02-10,23:42:53,javascript-i/functions,74,70.117.6.133,NaN,NaT,NaT,NaN
2018-02-10,23:47:26,javascript-i/functions,74,70.117.6.133,NaN,NaT,NaT,NaN


In [34]:
#looking through mid-sections of data
df[df.isnull().any(axis=1)][505:800]

,time,path,user_id,ip,name,start_date,end_date,program_id
date,,,,,,,,
2018-02-11,04:04:09,mysql,48,97.105.19.61,NaN,NaT,NaT,NaN
2018-02-11,04:04:12,mysql/tables,48,97.105.19.61,NaN,NaT,NaT,NaN
2018-02-11,04:07:57,mysql/databases,48,97.105.19.61,NaN,NaT,NaT,NaN
2018-02-11,06:41:28,spring,48,97.105.19.61,NaN,NaT,NaT,NaN
2018-02-11,06:41:33,spring/fundamentals/security,48,97.105.19.61,NaN,NaT,NaT,NaN
...,...,...,...,...,...,...,...,...
2018-02-28,19:30:32,javascript-i/loops,111,70.117.6.133,NaN,NaT,NaT,NaN
2018-02-28,21:00:18,javascript-i/loops,111,70.117.6.133,NaN,NaT,NaT,NaN
2018-03-02,14:59:30,/,48,66.69.62.219,NaN,NaT,NaT,NaN


In [54]:
#who dis? 
df[df.user_id == ]

,time,path,user_id,ip,name,start_date,end_date,program_id
date,,,,,,,,
2018-01-28,18:48:19,index.html,53,67.10.161.55,Lassen,2016-07-18,2016-11-10,1.0
2018-01-28,18:48:24,html-css,53,67.10.161.55,Lassen,2016-07-18,2016-11-10,1.0
2018-01-28,18:49:20,javascript-i,53,67.10.161.55,Lassen,2016-07-18,2016-11-10,1.0
2018-01-28,18:49:23,javascript-i/conditionals,53,67.10.161.55,Lassen,2016-07-18,2016-11-10,1.0
2018-01-28,18:50:41,javascript-i/conditionals,53,67.10.161.55,Lassen,2016-07-18,2016-11-10,1.0
...,...,...,...,...,...,...,...,...
2021-04-20,09:53:33,capstone/sw-project-planning,53,65.43.115.179,Staff,2014-02-04,2014-02-04,2.0
2021-04-20,11:14:25,java-iii,53,65.43.115.179,Staff,2014-02-04,2014-02-04,2.0
2021-04-20,11:14:26,java-iii/finish-the-adlister,53,65.43.115.179,Staff,2014-02-04,2014-02-04,2.0


In [ ]:
#who dis? (this is showing a staff being a staff prior to student? possible?)
df[df.user_id == 53]

In [18]:
#finding any missing names of cohorts (any possible scrapers?)
df[df.isnull().any(axis=1)].tail()

,time,path,user_id,ip,name,start_date,end_date,program_id
date,,,,,,,,
2021-04-21,12:49:00,javascript-ii,717,136.50.102.126,NaN,NaT,NaT,NaN
2021-04-21,12:49:02,javascript-ii/es6,717,136.50.102.126,NaN,NaT,NaT,NaN
2021-04-21,12:51:27,javascript-ii/map-filter-reduce,717,136.50.102.126,NaN,NaT,NaT,NaN
2021-04-21,12:52:37,javascript-ii/promises,717,136.50.102.126,NaN,NaT,NaT,NaN
2021-04-21,12:56:26,javascript-ii/RESTful-api,717,136.50.102.126,NaN,NaT,NaT,NaN


In [19]:
#who dis?
df[df.user_id == 717]

,time,path,user_id,ip,name,start_date,end_date,program_id
date,,,,,,,,
2020-07-20,16:16:37,/,717,136.50.87.115,NaN,NaT,NaT,NaN
2020-07-20,16:16:47,toc,717,136.50.87.115,NaN,NaT,NaT,NaN
2020-07-20,16:16:54,toc,717,136.50.87.115,NaN,NaT,NaT,NaN
2020-07-21,09:16:00,toc,717,136.50.87.115,NaN,NaT,NaT,NaN
2020-07-21,09:17:20,toc,717,136.50.87.115,NaN,NaT,NaT,NaN
...,...,...,...,...,...,...,...,...
2021-04-21,12:49:00,javascript-ii,717,136.50.102.126,NaN,NaT,NaT,NaN
2021-04-21,12:49:02,javascript-ii/es6,717,136.50.102.126,NaN,NaT,NaT,NaN
2021-04-21,12:51:27,javascript-ii/map-filter-reduce,717,136.50.102.126,NaN,NaT,NaT,NaN


In [20]:
#possibly webscraper...pages are being accessed only one day by this unknown user every minute
df[df.ip == '136.50.102.126']

,time,path,user_id,ip,name,start_date,end_date,program_id
date,,,,,,,,
2021-02-13,11:39:43,jquery/personal-site,717,136.50.102.126,NaN,NaT,NaT,NaN
2021-03-08,10:04:54,jquery,717,136.50.102.126,NaN,NaT,NaT,NaN
2021-03-08,10:05:12,capstone/workbook,717,136.50.102.126,NaN,NaT,NaT,NaN
2021-03-31,12:20:36,toc,717,136.50.102.126,NaN,NaT,NaT,NaN
2021-03-31,12:20:42,javascript-i/bom-and-dom/dom,717,136.50.102.126,NaN,NaT,NaT,NaN
2021-03-31,12:20:48,javascript-i/bom-and-dom/dom-events,717,136.50.102.126,NaN,NaT,NaT,NaN
2021-03-31,12:20:56,javascript-i/bom-and-dom/dom,717,136.50.102.126,NaN,NaT,NaT,NaN
2021-03-31,12:21:04,jquery,717,136.50.102.126,NaN,NaT,NaT,NaN
2021-03-31,12:21:15,jquery/effects,717,136.50.102.126,NaN,NaT,NaT,NaN


In [56]:
df.program_id.value_counts()

2.0    713365
3.0    103412
1.0     30548
4.0         5
Name: program_id, dtype: int64

In [21]:
df[df.path == '/'].value_counts()

time      path  user_id  ip              name     start_date  end_date    program_id
11:58:53  /     580      107.77.217.143  Curie    2020-02-03  2020-07-07  3.0           8
21:42:46  /     580      74.192.167.159  Curie    2020-02-03  2020-07-07  3.0           7
12:25:51  /     580      74.192.167.159  Curie    2020-02-03  2020-07-07  3.0           5
07:32:55  /     580      74.192.167.159  Curie    2020-02-03  2020-07-07  3.0           5
13:28:32  /     580      74.192.167.159  Curie    2020-02-03  2020-07-07  3.0           4
                                                                                       ..
10:08:07  /     579      136.50.64.245   Curie    2020-02-03  2020-07-07  3.0           1
10:08:08  /     4        97.105.19.61    Teddy    2018-01-08  2018-05-17  2.0           1
                252      4.16.216.161    Ulysses  2018-03-05  2018-07-19  2.0           1
                477      97.105.19.58    Bayes    2019-08-19  2020-01-30  3.0           1
23:59:58  /    

In [53]:
df2 = df.groupby(['user_id'])['name'].count()
df2.head()

user_id
1    7404
2    1541
3    1562
4     692
5    1701
Name: name, dtype: int64

In [55]:
df(df.program_id == 4)

TypeError: 'DataFrame' object is not callable

In [58]:
df.path.value_counts()[55:100
                      ]

javascript-ii/es6                              4669
spring/setup                                   4657
html-css/css-i/box-model                       4616
javascript-ii/ajax-api                         4607
java-i/control-statements-and-loops            4596
spring/fundamentals/relationships              4563
javascript-ii/map-filter-reduce                4537
jquery/essential-methods/attributes-and-css    4484
java-iii/user-input                            4465
jquery/introduction                            4408
html-css/introduction                          4395
jquery/events/mouse-events                     4395
jquery/ajax/requests-and-responses             4359
javascript-i/bom-and-dom/bom                   4317
mysql/relationships/joins                      4289
javascript-i/introduction/variables            4262
jquery/essential-methods/traversing            4195
jquery/effects                                 4040
spring/fundamentals/form-model-binding         4026
java-ii/file